In [ ]:
import os

os.chdir("../")

from pathlib import Path

import hydra
import matplotlib.pyplot as plt
import torch
from matplotlib import colormaps as cm

cmap = cm.get_cmap("viridis")

hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(version_base="1.3", config_path="../configs/")
cfg = hydra.compose(config_name="eval.yaml")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dm = hydra.utils.instantiate(cfg.data)
dm.setup("test")

dl = dm.test_dataloader()
ds = dm._data_test

In [ ]:
root_dir = Path("logs/train/runs")
ckpt_path = root_dir / "<checkpoint location>"

model = hydra.utils.instantiate(cfg.model)
checkpoint = torch.load(ckpt_path, map_location=device)
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

In [ ]:
for i in [2, 3, 4, 5]:
    item = ds[i]
    img = item["image"]
    plot_img = img.permute(1, 2, 0).numpy()
    mask = item["target_segmentation"]
    person_mask = mask[1]

    img = img.to(device)
    pred_mask = torch.sigmoid(model(img.unsqueeze(0))["seg_logits"][0]).detach().cpu().numpy()
    pred_person_mask = pred_mask[1]

    fig, ax = plt.subplots(1, 4, figsize=(20, 5))
    ax[0].imshow(plot_img)

    ax[1].imshow(person_mask, cmap="gray")

    ax[2].imshow(pred_person_mask, cmap="gray", vmin=0, vmax=1)

    thresholded_pred_person_mask = pred_person_mask > 0.5
    ax[3].imshow(thresholded_pred_person_mask, cmap="gray")

    plt.show()